## Actitracker labeled dataset NN model
This model will use keras to predict activity from sessionized accelerometer data, via the actitracker dataset (http://www.cis.fordham.edu/wisdm/dataset.php). 

### Load libraries

In [71]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy as sp
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
%matplotlib inline

### Data load

In [72]:
DATA_FOLDER = 'Data/WISDM_ar_v1.1/'
DATA_FILE = 'raw_data_fixed.txt'
actitracker = pd.read_csv(
    DATA_FOLDER+DATA_FILE ,
    sep=',' ,
    lineterminator=';' ,
    header=None ,
)
actitracker.columns = [
    'user' ,
    'activity' ,
    'timestamp' ,
    'x-accel' ,
    'y-accel' ,
    'z-accel' ,
    'NA' ,
]
del actitracker['NA']
accel_cols = ['x-accel','y-accel','z-accel']

### Sessionize data

In [73]:
# re-calculate time in seconds
actitracker['time_seconds'] = actitracker['timestamp']*10e-9

# sort by user and time 
actitracker = actitracker.sort_values(by=['user','time_seconds'])
actitracker['seq'] = xrange(actitracker.shape[0])
actitracker['session'] = actitracker.groupby(['user','activity'])['seq'].apply(lambda x: x%500 == 0).fillna(0).cumsum()

### Gather labels

In [74]:
# get session_labels 
ohe = OneHotEncoder(sparse=False); le = LabelEncoder()
labels = actitracker.groupby(['user','session'])['activity'].apply(lambda x: max(x))
ohe_labels = ohe.fit_transform(le.fit_transform(labels).reshape(-1,1))
Y = pd.DataFrame(ohe_labels,index=labels.index)

### Calculate features 

In [86]:
# group by user and session
accel_cols = ['x-accel','y-accel','z-accel']
g = actitracker.loc[:,accel_cols+['user','session']].groupby(['user','session'])

# calculate model cols
def iqr(x):
    ''' pass a series object 
        to calculate IQR
    '''
    q75, q25 = np.percentile(x, [75,25])
    return q75-q25

means = g[accel_cols].apply(lambda x: np.mean(x))
sds = g[accel_cols].apply(lambda x: np.std(x))
median_1 = g[accel_cols[0]].apply(lambda x: np.median(x))
median_2 = g[accel_cols[1]].apply(lambda x: np.median(x))
median_3 = g[accel_cols[2]].apply(lambda x: np.median(x))
iqr_1 = g[accel_cols[0]].apply(lambda x: iqr(x))
iqr_2 = g[accel_cols[1]].apply(lambda x: iqr(x))
iqr_3 = g[accel_cols[2]].apply(lambda x: iqr(x))
mins = g[accel_cols].apply(lambda x: np.min(x))
maxs = g[accel_cols].apply(lambda x: np.max(x))

# concat columns
X = pd.concat((means,
               sds,
               median_1,
               median_2,
               median_3,
               iqr_1,
               iqr_2,
               iqr_3,
               mins,
               maxs,
              ),axis=1)

### Neural Network Model

In [68]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from keras.layers.recurrent import LSTM

#### Compile model

In [91]:
model = Sequential()

indim = X.shape[1]
model.add(Dense(512*8, input_dim=indim, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512*12, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

#### Execute model

In [92]:
X_train, X_test, Y_train, Y_test = train_test_split(X.as_matrix(), Y.as_matrix(), test_size=0.33, random_state=22)
model.fit(X_train, Y_train,
          nb_epoch=20,
          batch_size=20)

Epoch 1/20
1495/1495 [==============================] - 15s - loss: 1.3590 - acc: 0.6107    
Epoch 2/20
1495/1495 [==============================] - 15s - loss: 0.8550 - acc: 0.7030    
Epoch 3/20
1495/1495 [==============================] - 15s - loss: 0.7523 - acc: 0.7318    
Epoch 4/20
1495/1495 [==============================] - 14s - loss: 0.7144 - acc: 0.7351    
Epoch 5/20
1495/1495 [==============================] - 15s - loss: 0.6799 - acc: 0.7605    
Epoch 6/20
1495/1495 [==============================] - 15s - loss: 0.6593 - acc: 0.7686    
Epoch 7/20
1495/1495 [==============================] - 15s - loss: 0.6485 - acc: 0.7666    
Epoch 8/20
1495/1495 [==============================] - 15s - loss: 0.6244 - acc: 0.7873    
Epoch 9/20
1495/1495 [==============================] - 14s - loss: 0.6223 - acc: 0.7779    
Epoch 10/20
1495/1495 [==============================] - 15s - loss: 0.6216 - acc: 0.7766    
Epoch 11/20
1495/1495 [==============================] - 14s - loss: 

In [94]:
predictions = model.predict(X_test)
print 'Accuracy: {}'.format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(predictions,axis=1)) )
print 'Log-loss: {}'.format(log_loss(Y_test, predictions))

Accuracy: 0.799185888738
Log-loss: 0.566244617757
